In [1]:
# ## install finrl library

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
%matplotlib inline
from finrl.config_tickers import SP_500_TICKER
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from settings import *
from pprint import pprint
import sys
import os
import itertools
import json

if not os.path.exists(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)

sys.path.append("../FinRL-Library")


In [4]:
INDICATORS = ['macd',
               'rsi_30',
               'cci_30',
               'dx_30']

with open("data/stock.json") as f:
    list_tickers = list(json.load(f).keys())

In [5]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = list_tickers).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Shape of DataFrame:  (251700, 8)


In [6]:
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

Successfully added technical indicators
Successfully added turbulence index


In [7]:
stock_dimension = len(processed.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 50, State Space: 301


In [8]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "print_verbosity":5

}

In [12]:
rebalance_window = 63 # rebalance_window is the number of days to retrain the model
validation_window = 63 # validation_window is the number of days to do validation and trading (e.g. if validation_window=63, then both validation and trading period will be 63 days)

In [23]:
ensemble_agent = DRLEnsembleAgent(df=processed,
                 train_period=(TRAIN_START_DATE,TRAIN_END_DATE),
                 val_test_period=(TEST_START_DATE,TEST_END_DATE),
                 rebalance_window=rebalance_window,
                 validation_window=validation_window,
                 **env_kwargs)

In [24]:
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.005,
                    'learning_rate': 0.0007
                    }

PPO_model_kwargs = {
                    "ent_coef":0.01,
                    "n_steps": 2048,
                    "learning_rate": 0.00025,
                    "batch_size": 128
                    }

DDPG_model_kwargs = {
                      #"action_noise":"ornstein_uhlenbeck",
                      "buffer_size": 10_000,
                      "learning_rate": 0.0005,
                      "batch_size": 64
                    }

SAC_model_kwargs = {
    "batch_size": 64,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

TD3_model_kwargs = {"batch_size": 100, "buffer_size": 1000000, "learning_rate": 0.0001}




timesteps_dict = {'a2c' : 10_000,
                 'ppo' : 10_000,
                 'ddpg' : 10_000,
                 'sac' : 10_000,
                 'td3' : 10_000
}

In [47]:
df_summary = ensemble_agent.run_ensemble_strategy(A2C_model_kwargs,
                                                 PPO_model_kwargs,
                                                 DDPG_model_kwargs,
                                                 SAC_model_kwargs,
                                                 TD3_model_kwargs,
                                                 timesteps_dict)

============Start Ensemble Strategy============
turbulence_threshold:  330.749240787695
======Model training from:  2002-01-01 to  2018-01-02
======a2c Training========
{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0007}
Using cpu device
Logging to tensorboard_log/a2c/a2c_126_2
-------------------------------------
| time/                 |           |
|    fps                | 51        |
|    iterations         | 100       |
|    time_elapsed       | 9         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -71       |
|    explained_variance | 0.205     |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | 25        |
|    reward             | 2.0841537 |
|    std                | 1         |
|    value_loss         | 0.246     |
-------------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 

/home/hainam/miniconda3/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.62GB > 1.24GB
  warnings.warn(


======td3 Validation from:  2018-01-02 to  2018-04-04
td3 Sharpe Ratio:  -0.05505769216932544
======sac Training========
{'batch_size': 64, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to tensorboard_log/sac/sac_126_1
day: 4027, episode: 10
begin_total_asset: 1000000.00
end_total_asset: 3421489.37
total_reward: 2421489.37
total_cost: 252128.65
total_trades: 138462
Sharpe: 0.492
======sac Validation from:  2018-01-02 to  2018-04-04
sac Sharpe Ratio:  -0.027732103563941506
======ppo Training========
{'ent_coef': 0.01, 'n_steps': 2048, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Logging to tensorboard_log/ppo/ppo_126_1
-----------------------------------
| time/              |            |
|    fps             | 58         |
|    iterations      | 1          |
|    time_elapsed    | 35         |
|    total_timesteps | 2048       |
| train/             |            |
|    reward          | 0.21235

/home/hainam/miniconda3/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.62GB > 1.11GB
  warnings.warn(


======td3 Validation from:  2018-04-04 to  2018-07-03
td3 Sharpe Ratio:  0.07641877340852711
======sac Training========
{'batch_size': 64, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to tensorboard_log/sac/sac_189_1
day: 4090, episode: 10
begin_total_asset: 1000000.00
end_total_asset: 5172149.91
total_reward: 4172149.91
total_cost: 63617.11
total_trades: 114726
Sharpe: 0.589
======sac Validation from:  2018-04-04 to  2018-07-03
sac Sharpe Ratio:  0.1012075099090205
======ppo Training========
{'ent_coef': 0.01, 'n_steps': 2048, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Logging to tensorboard_log/ppo/ppo_189_1
-----------------------------------
| time/              |            |
|    fps             | 53         |
|    iterations      | 1          |
|    time_elapsed    | 37         |
|    total_timesteps | 2048       |
| train/             |            |
|    reward          | 0.22314787 |

...

======td3 Validation from:  2020-10-02 to  2021-01-04
td3 Sharpe Ratio:  0.46618150870654795
======sac Training========
{'batch_size': 64, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to tensorboard_log/sac/sac_819_1
day: 4720, episode: 10
begin_total_asset: 1000000.00
end_total_asset: 6862744.34
total_reward: 5862744.34
total_cost: 126508.01
total_trades: 133569
Sharpe: 0.537
======sac Validation from:  2020-10-02 to  2021-01-04
sac Sharpe Ratio:  0.16263423156652954
======ppo Training========
{'ent_coef': 0.01, 'n_steps': 2048, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Logging to tensorboard_log/ppo/ppo_819_1
-----------------------------------
| time/              |            |
|    fps             | 44         |
|    iterations      | 1          |
|    time_elapsed    | 45         |
|    total_timesteps | 2048       |
| train/             |            |
|    reward          | 0.05906761

/home/hainam/miniconda3/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.62GB > 0.58GB
  warnings.warn(


======td3 Validation from:  2021-01-04 to  2021-04-06
td3 Sharpe Ratio:  0.2668946462825803
======sac Training========
{'batch_size': 64, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to tensorboard_log/sac/sac_882_1
day: 4783, episode: 10
begin_total_asset: 1000000.00
end_total_asset: 6169029.00
total_reward: 5169029.00
total_cost: 444110.88
total_trades: 186726
Sharpe: 0.567
======sac Validation from:  2021-01-04 to  2021-04-06
sac Sharpe Ratio:  0.2018262805599679
======ppo Training========
{'ent_coef': 0.01, 'n_steps': 2048, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Logging to tensorboard_log/ppo/ppo_882_1
-----------------------------------
| time/              |            |
|    fps             | 43         |
|    iterations      | 1          |
|    time_elapsed    | 46         |
|    total_timesteps | 2048       |
| train/             |            |
|    reward          | 0.46874663 |

/home/hainam/miniconda3/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.62GB > 0.50GB
  warnings.warn(


======td3 Validation from:  2021-04-06 to  2021-07-06
td3 Sharpe Ratio:  0.6035980634900817
======sac Training========
{'batch_size': 64, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to tensorboard_log/sac/sac_945_1
day: 4846, episode: 10
begin_total_asset: 1000000.00
end_total_asset: 12496611.84
total_reward: 11496611.84
total_cost: 363856.64
total_trades: 153832
Sharpe: 0.701
======sac Validation from:  2021-04-06 to  2021-07-06
sac Sharpe Ratio:  0.30264427969567476
======ppo Training========
{'ent_coef': 0.01, 'n_steps': 2048, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Logging to tensorboard_log/ppo/ppo_945_1
-----------------------------------
| time/              |            |
|    fps             | 42         |
|    iterations      | 1          |
|    time_elapsed    | 48         |
|    total_timesteps | 2048       |
| train/             |            |
|    reward          | 0.4216107